In [1]:
from nltk.corpus import wordnet
from fuzzywuzzy import fuzz, StringMatcher
import csv
import nltk
import re
import string
import pandas as pd
import zipfile
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [2]:
class Tokenizer:
    def __cleaned_words(self, sentence):
        words = re.split(r'\W+', sentence)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in words]
        final_words=[]
        for word in stripped:
            if(word is not ''):
                final_words.append(word)
        return final_words
    
    def tokenize(self, sentence):
        tokens = self.__cleaned_words(sentence)
        return tokens
    

In [3]:
tokenizer = Tokenizer()

In [4]:
tokenizer.tokenize("My Name isn't Guru")

['My', 'Name', 'isn', 't', 'Guru']

Alright, so the tokenizer meha implemented is definitely not ideal. It should not be splitting `isn't` to `isn` and `t`. We're definitely losing information here. Let's see how nltk does it before proceeding.

In [5]:
word_tokenize("My Name isn't Guru")

['My', 'Name', 'is', "n't", 'Guru']

This is better. This would work if `n't` and `not` are similar in the word vectors. 

In [22]:
def GetGlove(glove_file):
    with open(glove_file, 'r') as file:
        words = set()
        word_to_vec, word_to_index, index_to_word = {},{},{}

        for line in file:
            line = line.strip().split()
            words.add(line[0])
            word_to_vec[line[0]] = np.array(line[1:], dtype='float64')
    
    for x,y in enumerate(sorted(words)):
        word_to_index[y] = x+1
        index_to_word[x+1] = y
        
    return {'word_to_vec': word_to_vec, 'word_to_index': word_to_index, 'index_to_word': index_to_word}


In [23]:
params = GetGlove('glove.6B.100d.txt')
word_to_vec, word_to_index, index_to_word = params['word_to_vec'],params['word_to_index'],params['index_to_word']

In [9]:
def cosine_similarity(u, v):
    """
    Cosine similarity reflects the degree of similariy between u and v
        
    Arguments:
        u -- a word vector of shape (n,)          
        v -- a word vector of shape (n,)

    Returns:
        cosine_similarity -- the cosine similarity between u and v defined by the formula above.
    """
    
    distance = 0.0
    dot = np.sum(u * v)
    norm_u = np.sqrt(np.sum(u ** 2))
    norm_v = np.sqrt(np.sum(v ** 2))
    cosine_similarity = dot / (norm_u * norm_v)

    return cosine_similarity

In [10]:
cosine_similarity(word_to_vec['n\'t'], word_to_vec['not'])

0.9010410922621748

They are. What about other punctuations like `Guru's`.

In [11]:
word_tokenize("Guru's food")

['Guru', "'s", 'food']

I guess this wouldn't make as much of a difference to the rnn. We could remove it along with the stopwords or just leave it be. I think it's fine tbh.

In [12]:
cosine_similarity(word_to_vec['\'s'], word_to_vec['has'])

0.7748113832696024

We should definitely leave it be. The glove model learnt it's own shit lol

In [16]:
docs = ["my name is guru", "I love food"]

In [17]:
docs

['my name is guru', 'I love food']

In [20]:
word_tokenize(docs[1])

['I', 'love', 'food']

In [24]:
index_to_word[0]

KeyError: 0

In [25]:
word_to_index[0]

KeyError: 0